In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from kernels import *
import pdb
import gpytorch
from itertools import product

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
train_x = torch.linspace(float(0), float(1), int(50))
one = torch.sin(train_x * (float(2) * math.pi)) + torch.randn(train_x.size()) * float(0.2)
two = torch.cos(train_x * (float(2) * math.pi)) + torch.randn(train_x.size()) * float(0.2)
train_y = torch.stack([one, two], int(-1))


In [23]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ZeroMean(), num_tasks=2
        )
        kernel = Diff_SE_kernel()
        kernel2 = Diff_SE_kernel()
        q, dx1, dx2 = var('q, dx1, dx2')
        # TODO test what happens with 
        #L = matrix(2, 2, (dx1, q, 0, dx1))
        # -> does it learn q as a parameter?
        #AND
        #L = matrix(2, 2, (q*dx1, q, 0, dx1))
        # -> does it learn multiple separate q?
        L = matrix(2, 2, (q*dx1, 0, 0, dx1))
        R = matrix(2, 2, (dx2, 0, 0, dx2))
        p = DiffMatrixKernel([[kernel, None], [None, kernel2]])
        self.covar_module = p.diff(left_matrix=L, right_matrix=R)
        #kernel0 = Diff_SE_kernel()
        #kernel1 = Diff_SE_kernel()
        #kernel2 = Diff_SE_kernel()
        #self.covar_module = MatrixKernel([[kernel0, None], [None, kernel2]])

    def forward(self, x):
        #pdb.set_trace()
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        #print(f"{covar_x.detach().evaluate()}")
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x, validate_args=True)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=2)
model = MultitaskGPModel(train_x, train_y, likelihood)

> /Users/andreas/Documents/container_storage/sage/DiffEqGPs/kernels.py(290)prepare_asym_deriv_dict()
    288                         import pdb
    289                         pdb.set_trace()
--> 290                         degr, coeff = self.single_term_extract(d_poly, d_var)
    291                     # If the operand does not contain a "d" it is a constant
    292                     # TODO: This should never happen since the first if

ipdb> c
> /Users/andreas/Documents/container_storage/sage/DiffEqGPs/kernels.py(203)single_term_extract()
    201         pdb.set_trace()
    202         # See if it's of the form a*x^n
--> 203         if (not len(d_poly.operands()) == 0) and ('^' in str(d_poly.operands()[0]) or '^' in str(d_poly.operands()[1])):
    204             # 1 if the coefficient is in [1], else it must be in [0]
    205             coeff_index = int('^' in str(d_poly.operands()[0]))

ipdb> d_poly.operands()
[dx1, q]
ipdb> not len(d_poly.operands()) == 0
True
ipdb> q


BdbQuit: 

In [8]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = int(2) if smoke_test else int(20)


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=float(0.1))  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    #print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f  variance: %.3f noise: %.3f' % (
    #    i + 1, training_iter, loss.item(),
    #    model.covar_module.length.item(),
    #    model.covar_module.var.item(),
    #    model.likelihood.noise.item()
    #))
    optimizer.step()

tensor([[1.0000, 0.9994, 0.9975,  ..., 0.0505, 0.0250, 0.0000],
        [0.9994, 1.0000, 0.9994,  ..., 0.0764, 0.0505, 0.0250],
        [0.9975, 0.9994, 1.0000,  ..., 0.1027, 0.0764, 0.0505],
        ...,
        [0.0505, 0.0764, 0.1027,  ..., 1.0000, 0.9994, 0.9975],
        [0.0250, 0.0505, 0.0764,  ..., 0.9994, 1.0000, 0.9994],
        [0.0000, 0.0250, 0.0505,  ..., 0.9975, 0.9994, 1.0000]],
       grad_fn=<CatBackward>)
tensor([[1.3580, 1.3570, 1.3538,  ..., 0.1219, 0.1035, 0.0854],
        [1.3570, 1.3580, 1.3570,  ..., 0.1404, 0.1219, 0.1035],
        [1.3538, 1.3570, 1.3580,  ..., 0.1590, 0.1404, 0.1219],
        ...,
        [0.1219, 0.1404, 0.1590,  ..., 1.3580, 1.3570, 1.3538],
        [0.1035, 0.1219, 0.1404,  ..., 1.3570, 1.3580, 1.3570],
        [0.0854, 0.1035, 0.1219,  ..., 1.3538, 1.3570, 1.3580]],
       grad_fn=<CatBackward>)
tensor([[1.6627, 1.6612, 1.6567,  ..., 0.1412, 0.1267, 0.1122],
        [1.6612, 1.6627, 1.6612,  ..., 0.1559, 0.1412, 0.1267],
        [1.6567,

In [9]:
for p in model.named_parameters():
    print(p)

('likelihood.raw_task_noises', Parameter containing:
tensor([-2.0469, -2.0387], requires_grad=True))
('likelihood.raw_noise', Parameter containing:
tensor([-2.0440], requires_grad=True))
('covar_module.kernel_00.kernels.0.var', Parameter containing:
tensor(0.5911, requires_grad=True))
('covar_module.kernel_00.kernels.0.length', Parameter containing:
tensor(0.4500, requires_grad=True))
('covar_module.kernel_00.kernels.1.var', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_00.kernels.1.length', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_00.kernels.2.var', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_00.kernels.2.length', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_00.kernels.3.var', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_00.kernels.3.length', Parameter containing:
tensor(1., requires_grad=True))
('covar_module.kernel_01.kernels.

In [ ]:
tuple([1,2])

In [ ]:
# Set into eval mode
model.eval()
likelihood.eval()

# Initialize plots

number_of_samples = int(50)
# Make predictions
with torch.no_grad():#, gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(float(0), float(2), number_of_samples)
    #pdb.set_trace()
    outputs = model(test_x)
    predictions = likelihood(outputs)
    
    mean = predictions.mean
    lower, upper = predictions.confidence_region()
#print(mean)
#print(lower)
#print(upper)
# This contains predictions for both tasks, flattened out
# The first half of the predictions is for the first task
# The second half is for the second task

#dims = int(2)
#indices = [list(range(i, len(train_y), dims)) for i in range(dims)]


In [ ]:
f, (y1_ax, y2_ax) = plt.subplots(int(1), int(2), figsize=(int(8), int(3)))

# Plot training data as black stars
y1_ax.plot(train_x.detach().numpy(), train_y[:, 0].detach().numpy(), 'k*')
# Predictive mean as blue line
y1_ax.plot(test_x.numpy(), mean[:, 0].numpy(), 'b')
# Shade in confidence
y1_ax.fill_between(test_x.numpy(), lower[:, 0].numpy(), upper[:, 0].numpy(), alpha=0.5)
y1_ax.set_ylim([-3, 8])
y1_ax.legend(['Observed Data', 'Mean', 'Confidence'])
y1_ax.set_title('Observed Values (Likelihood)')

# Plot training data as black stars
y2_ax.plot(train_x.detach().numpy(), train_y[:, 1].detach().numpy(), 'k*')
# Predictive mean as blue line
y2_ax.plot(test_x.numpy(), mean[:, 1].numpy(), 'b')
# Shade in confidence
y2_ax.fill_between(test_x.numpy(), lower[:, 1].numpy(), upper[:, 1].numpy(), alpha=0.5)
y2_ax.set_ylim([-3, 8])
y2_ax.legend(['Observed Data', 'Mean', 'Confidence'])
y2_ax.set_title('Observed Values (Likelihood)')


# Test Diffable SE Kernel

In [ ]:
X = torch.tensor([int(1), int(2), int(3)])
X

In [ ]:
x1, x2, l, sigma = var('x1, x2, l, sigma')
lengthscale = 1
variance = 1
SE(x1, x2, l, sigma) = sigma^2*exp(-(x1-x2)^2/(2*l^2))
cov_matr = [[None for i in range(len(X))] for j in range(len(X))]
for i, (v1, v2) in enumerate(product(X, X)):
    cov_matr[int(i/len(X))][int(i%len(X))] = SE.diff(x2).diff(x1)(int(v1), int(v2), lengthscale, variance)
cov_matr

In [ ]:
SE.operands()

In [ ]:
a = Diff_SE_kernel(var=int(variance), length=int(lengthscale))
q, dx1, dx2 = var('q, dx1, dx2')
left_poly = dx2
right_poly = dx1
diffed_kernel = a.diff(left_poly=left_poly, right_poly=right_poly, left_d_var=var('dx2'), right_d_var=var('dx1'))
diffed_kernel(X).evaluate()

In [ ]:
def calc_cell_diff(L, M, R, row, col):
    len_M = M.number_of_arguments()
    temp = None
    for j in range(int(sqrt(len_M))):
        if temp == None:
            import itertools
            #M_tr = list(map(list, itertools.zip_longest(*M, fillvalue=None)))
            #[M_tr[j].diff(left_poly=L[row][k], right_poly=R.transpose()[col][j]) for k in range(L.number_of_arguments())]
            temp = L[row]*M.transpose()[j]*R.transpose()[col][j]
        else:
            temp += L[row]*M.transpose()[j]*R.transpose()[col][j]
    return temp

In [ ]:
dimension = 2
length = dimension*dimension +1
L_list = [var(f'l_{i}{j}') for i in range(1, dimension+1) for j in range(1, dimension+1)]
M_list = [var(f'm_{i}{j}') for i in range(1, dimension+1) for j in range(1, dimension+1)]
R_list = [var(f'r_{i}{j}') for i in range(1, dimension+1) for j in range(1, dimension+1)]
L = matrix(dimension, dimension, L_list)
M = matrix(dimension, dimension, M_list)
R = matrix(dimension, dimension, R_list)
print(L)
print(M)
print(R)
row = 1
col = 0
print((L*M*R)[row][col])

calc_cell_diff(L, M, R, row, col)

In [ ]:
from itertools import product
for p in product(L.rows(),R.columns()):
    print(p)

In [ ]:
MSE(x1, x2, sigma, l) = matrix(2,2, (sigma^2*exp(-(x1-x2)^2/(2*l^2)), 0, 0, sigma^2*exp(-(x1-x2)^2/(2*l^2))))
dx1 = matrix(2,2,(dx1, 0, 0, dx1))
MSE

In [ ]:
kernel = Diff_SE_kernel()
kernel2 = Diff_SE_kernel()

p = DiffMatrixKernel([[kernel, None], [None, kernel2]])

In [ ]:
q, dx1, dx2 = var('q, dx1, dx2')
left_poly = dx1
right_poly = dx2
L = matrix(2, 2, (dx1, 0, 0, dx1))
R = matrix(2, 2, (dx2, 0, 0, dx2))

In [ ]:
p.diff(left_matrix=L, right_matrix=R).forward(X, X)

In [41]:
w, q, dx1, dx2 = var('w, q, dx1, dx2')
a = 3*w*dx1^2
#a.degree(dx1)
a.operands()

[dx1^2, w, 3]